The object here is to train a machine learning model on a database of Tweets and use that model to predict sentiment of Tweets from a user, a hashtag, or any random variety of Tweets. 

@author Ryan Herren
@author Tanner Dunn

In [21]:
import tweepy
import pandas as pd
import numpy as np
import json
import os
import yaml

In [22]:
os.environ['python-bot-config'] = "/Users/ryanherren/python-bot-config.yaml"
# os.environ['python-bot-config'] = "/Users/tcdunn/python-bot-config.yaml"
# os.environ['python-bot-config'] = "/Users/mariolgw/python-bot-config.yaml"

In [26]:
try:
    v_env = os.environ['python-bot-config']
except:
    print("Config file env variable is not set.")
    print("Set python-bot-config file")
    sys.exit(1)

with open(v_env, "r") as yamlConfig:
    cfg = yaml.safe_load(yamlConfig)

In [27]:
# consumer_key = 'vs3T1V8OD8fjMcNWsmtG43ylH'
# consumer_secret = 'KXBSHGpBZ3QQrqTFpVIOdghqNs6bzSyN0iqSAKUdi3vkjATNhp'
# access_token = '1463243342570037249-01yUhVcZaBWWrTbSMsIw4iyApKMxvA'
# access_token_secret = 'QywOsttbZtXwcdEoK05f6QjKH3uEbi4r0vWkIkSeQwB5f'

consumer_key = cfg.get("TwitterAPI").get("consumer_key")
consumer_secret = cfg.get("TwitterAPI").get("consumer_secret")
access_token = cfg.get("TwitterAPI").get("access_token")
access_token_secret = cfg.get("TwitterAPI").get("access_token_secret")

In [28]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
 
api = tweepy.API(auth, wait_on_rate_limit=True)

In [29]:
# # Iterates through and writes individual objects to the file, doesn't work when parsing file later
# f = open("tweets.json", "w")
# for tweet in tweepy.Cursor(api.home_timeline).items(10):
#     json.dump(tweet._json, f)
# f.close()

In [44]:
# # Evaluates all Tweets on a the bot's home timeline
# # Dumps raw json
# api.home_timeline(items=10)

In [37]:
# Currently limited to 10 Tweets
def get_tweets_by_user(user):
    tweets = api.user_timeline(user, items=10)
    my_list_of_dicts = []
    for each_json_tweet in tweets:
        my_list_of_dicts.append(each_json_tweet._json)
    filename = user + '.txt'
    with open(filename, 'w') as file:
        file.write(json.dumps(my_list_of_dicts, indent=4))
    my_demo_list = []
    with open(filename, encoding='utf-8') as json_file:  
        all_data = json.load(json_file)
        for each_dictionary in all_data:
            tweet_id = each_dictionary['id']
            text = each_dictionary['text']
            favorite_count = each_dictionary['favorite_count']
            retweet_count = each_dictionary['retweet_count']
            created_at = each_dictionary['created_at']
            my_demo_list.append({'tweet_id': str(tweet_id),
                                 'text': str(text),
                                 'favorite_count': int(favorite_count),
                                 'retweet_count': int(retweet_count),
                                 'created_at': created_at,
                                })
            #print(my_demo_list)
            tweet_json = pd.DataFrame(my_demo_list, columns = 
                                      ['tweet_id', 'text', 
                                       'favorite_count', 'retweet_count', 
                                       'created_at'])
    return tweet_json

In [41]:
ryan_tweets = get_tweets_by_user('ryanherren48')

In [42]:
ryan_tweets.head(10)

,tweet_id,text,favorite_count,retweet_count,created_at
0,1462910682446417927,"RT @CycloneWBB: 4-0, 𝙖𝙣𝙙 𝙞𝙩 𝙛𝙚𝙚𝙡𝙨 𝙨𝙤 𝙜𝙤𝙤𝙙.\n\n...",0,4,Mon Nov 22 22:27:40 +0000 2021
1,1462512722348855297,RT @CycloneWBB: Nothin' like a road win with t...,0,14,Sun Nov 21 20:06:19 +0000 2021
2,1458307529482588162,RT @sixers: https://t.co/aD22KOvIqj,0,703,Wed Nov 10 05:36:23 +0000 2021
3,1455733675086209025,RT @McDonalds: i present to u the most importa...,0,95039,Wed Nov 03 03:08:49 +0000 2021
4,1452352048847106060,🎥🎥🎥 https://t.co/PcsV8SmqNy,3,0,Sun Oct 24 19:11:26 +0000 2021
5,1443266190797938705,RT @TheSmarmyBum: Here’s a sneak peek👀 on what...,0,5675,Wed Sep 29 17:27:29 +0000 2021
6,1440524360821854224,RT @CycloneWBB: All about the process. \n\n#Io...,0,8,Wed Sep 22 03:52:25 +0000 2021
7,1439460383874502663,Fun fact of the day: Iowa State Punter Corey D...,6,0,Sun Sep 19 05:24:34 +0000 2021
8,1437836753872773128,RT @CycloneMBB: Lights always shine brighter a...,0,11,Tue Sep 14 17:52:50 +0000 2021
9,1433882660078772224,RT @CycloneWBB: Practice is where the magic st...,0,9,Fri Sep 03 20:00:41 +0000 2021
